# Week Five: Integration and differentiation

To be graded, your notebook must be runnable start to finish. If you can't make an in-notebook test pass, comment it out for to attempt to get partial credit. You should replace the `...` markers with your code. Do not change the names of the pre-defined ALL_CAPS variables and functions. (If you start from scratch, make sure you match the requested function names and requested ALL_CAPS variables). Other that that, you are free to define or make anything you wish. Remember that functions will often be tested with different numbers than the ones you are given.

In [20]:
# EID is your 6+2 UC Electronic ID
EID = '800215825'
NAME = 'Matthew Lazo'

In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Problem 1: 2D Trapezoidal rule

You can extend the trapezoidal rule to 2D. I'll give you a few hints:

The weight matrix is:
$$
W_{ij} = \begin{matrix}
1 & 2 & 2 & \cdots & 2 & 2 & 1 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
1 & 2 & 2 & \cdots & 2 & 2 & 1
\end{matrix}
$$

If you use the trick I used in the slides instead of explicitly creating this matrix, you would split the sum into 4 parts. However, it's really quite easy to make the matrix above

The "bin" width $h$ is joined by the 2nd dimension bin width of $k$, and then the trapezoidal rule integral value is 

$$
I_\mathrm{tr} = \frac{1}{4} h k \sum_i \sum_j W_{ij} f(x_{i}, y_{j})
$$

Write a function that takes a 2-parameter function 'f' and integrates it with the given parameters (further explanations are built into the docstring of the function). I'm writing a Numpy-style docstring.

See also: <http://mathfaculty.fullerton.edu/mathews/n2003/SimpsonsRule2DMod.html>

In [22]:
def trap_2d(f, xrange, yrange, xN, yN):
    '''Integrate a 2D function with the trapezoidal rule.
    
    This takes a 2D function and integrates it over a 2D grid.
    This uses the Trapezoidal rule in 2D.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take an x and y value (possibly arrays).
    xrange : (float, float)
        The min and max values for x, as an array
    yrange : (float, float)
        The min and max values for y, as an array
    xN : int
        The number of evaluations along x
    yN : int
        The number of evaluations along x

    Returns
    -------
    float
        The total integral
    
    '''
    # If these are *not* arrays, convert them into arrays. Otherwise, leave them alone.
    # (If we used np.array, we'd needlessly make a copy if it aready was an array)
    xrange = np.asarray(xrange)
    yrange = np.asarray(yrange)
    
    # There are N-1 "bins"
    h = (xrange[1] - xrange[0])/(xN-1) # x step size
    k = (yrange[1] - yrange[0])/(yN-1) # y step size
    
    weights = np.full((xN, yN),4)
    for i in range(yN):
      for j in range(xN):
        if i == 0 or i == yN-1 or j == 0 or j == xN-1:
          weights[i,j] = 2
    weights[0,0] = 1
    weights[yN-1,0] = 1
    weights[yN-1,xN-1] = 1
    weights[0,yN-1] = 1

    sum = 0
    x = 0
    y = 0
    for i in range(0, xN):
      for j in range(0, yN):
        sum += weights[i,j] * f(x,y)
        y += k
      x += h
    
    print(0.25 * h * k * sum)
    return 0.25 * h * k * sum


Feel free to add more tests, I'm adding the simplest one I can think of:

In [23]:
def f(x,y):
    return 1

def f1(x,y):
  return -x**2

def f2(x,y):
  return x * y**2

def f3(x,y):
  return np.exp(x)

In [24]:
area = trap_2d(f, [0, 10], [0, 10], 10, 10)
assert np.isclose(area, 100)

area2 = trap_2d(f1, [0, 10], [0, 10], 100, 100)
assert np.isclose(area2, -3333, rtol=0.5)

area4 = trap_2d(f3, [0, 10], [0, 10], 1000, 1000)
assert np.isclose(area4, 220254, rtol=2.5)

r = np.random.random(100)
r2 = (np.random.random(100) - 0.5) * 2 * 50
print(np.max(r), np.min(r))
print(np.max(r2), np.min(r2))

100.00000000000001
-3333.5033840084243
220256.49707695513
0.9566369480970123 0.011096388133265966
49.26694197131105 -49.01851254712327


# Problem 2: MC integration

Write a function that performs MC integration of an unknown function of N dimensions. You can use `f(*array)` to call the function with the correct number of arguments (it expands to `f(array[0], array[1], ..., array[N])`). Return **two** values; the estimated integral and the estimated error on the integral.

In [28]:
def mc_integrate(f, N, samples, ranges):
    '''
    Integrate an ND function with a set of sampled MC points.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take N
        values (possibly equal length arrays).
    N : int
        The min and max values for x, as an array
    samples : int
        The number of samples to integrate over
    ranges : array[2, N]
        An array with the min and max for each dimension.

    Returns
    -------
    float
        The total integral
    float
        The estimated error = relative error * the total integral.
    '''
    
    ranges = np.asarray(ranges) # Just making sure ranges is an array
    
    # Hint: You can turn a 1D array, such as arr = array([a, b]) into a 2D array
    # with arr[:,np.newaxis] or arr[:,None] or arr.reshape(2,1) or arr.reshape(-1,1)
    
    # Our definition of ranges matches what you find in Scipy, and is natural to type,
    # but is a but less natural to use, for example:
    # widths = ranges[:,1] + ranges[:,0]

    r = np.random.random(samples)
    points = np.zeros(0)
    print(points.shape)
    for i in range(N):
      points = points[np.newaxis]
      print(points.shape)
      points[:,i] = r[::(len(r)/2-1)]
    print(points.shape)

    error = eval / np.sqrt(N)

In [12]:
def f(x,y):
    return x + y

In [29]:
integral, error = mc_integrate(f, 2, 10_000, [[0,1],[0,1]])

(0,)
(1, 0)


TypeError: ignored

In [ ]:
print(integral, error)

#### For comparison, we could use NQuad from Scipy:

In [ ]:
from scipy import integrate

In [ ]:
integrate.nquad(f, [[0,1],[0,1]])

# Problem 3: Second derivatives

Write a function that calculates:

$$
\frac{d^2 f(t)}{dt^2} \Biggr\rvert_\mathrm{cd}
=
\frac{
\left[
f(t + h) - f(t)
\right] - \left[
f(t) - f(t - h)
\right]
}{h^2}
$$

(The notes in class had a typo, fixed here.) Write a nice docstring as well.

In [25]:
def d2fdt2(f, t, h):
  ''' Returns the numerical value of the second derivative of the provided
        function, where f is the function f(t), t is the value at which the
        second derivative is evaluated, and h is the numerical step size used
        to calculate the second derivative using the central difference
        definition of the derivative.

  Parameters
  ----------
  f : callable
    The function to differentiate.
  t : float
    The value at which the second derivative of f will be evaluated.
  h : float
    The step size used in the central difference definition of the derivative.

  Returns
  -------
  float
    The value of the second derivative of f at t.
  '''
  try:
    ypp = ( (f(t+h) - f(t)) - (f(t) - f(t-h)) ) / h**2
  except: # Intended to stop user from calling an undefined function, though this seems to be handled by Python anyway
          # (uncomment the final line in the code block below; 'aaa' will not print so this except is not used).
    print('aaa')
    raise NameError()
  
  return ypp

Test your `d2fdt2` function on a function that you know the derivative for.

In [26]:
def cube(x):
  return x**3

def exp_x2(x):
  return np.exp(x**2) 

a = d2fdt2(cube, 1, 0.01)
print('Second derivative of f(x) = x^3 is 6x.')
print('d2fdt2(cube, 1, 0.01) = ', a)
print('Compared to 6 from hand calculation.\n')

b = d2fdt2(np.sin, np.pi/2, 0.01)
print('Second derivative of f(x) = sin(x) is -sin(x).')
print('d2fdt2(sin, 1, 0.01) = ', b)
print('Compared to numpy, -sin(pi/2) =', -np.sin(np.pi/2), ' or -1 analytically.\n')

c = d2fdt2(exp_x2, 1, 0.01)
print('Second derivative of f(x) = exp(x^2) is [2*exp(x^2)] * (1 + 2*(x^2)).')
print('d2fdt2(exp_x2, 1, 0.01) = ', c)
print('Compared to 16.30969097 from scientific calculator.')

Second derivative of f(x) = x^3 is 6x.
d2fdt2(cube, 1, 0.01) =  6.00000000000156
Compared to 6 from hand calculation.

Second derivative of f(x) = sin(x) is -sin(x).
d2fdt2(sin, 1, 0.01) =  -0.9999916666947328
Compared to numpy, -sin(pi/2) = -1.0  or -1 analytically.

Second derivative of f(x) = exp(x^2) is [2*exp(x^2)] * (1 + 2*(x^2)).
d2fdt2(exp_x2, 1, 0.01) =  16.311412653755575
Compared to 16.30969097 from scientific calculator.
